In [1]:
import torch
from torch import nn as nn
import torchvision
from torch.utils.data import DataLoader, dataset
from torchvision.transforms import AutoAugment, Normalize, Compose, ToTensor
import torch.nn.functional as F
from torchmetrics import Accuracy
import numpy as np

In [2]:
class accuracy:
    def __init__(self):
        pass
    def update(self, x,y):
        pass
    def reset(self):
        pass

In [35]:
def __normalize__(x):
    return x/255. -1/2    #####Normalization is different here!
normalize = torch.jit.script(__normalize__)

In [36]:
from torchvision.datasets import CIFAR100

In [37]:
data = CIFAR100("./data", train = True, download = True, transform = Compose([AutoAugment(),ToTensor(), normalize]))
test_data = CIFAR100("./data", train = False, download = True, transform = Compose([ToTensor(),  normalize]))

Files already downloaded and verified
Files already downloaded and verified


In [38]:
train_set = DataLoader(data, 128, shuffle = True, num_workers=2)
test_set = DataLoader(test_data, 128, shuffle = False, num_workers=2)

In [39]:
class VGG5(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_1 = nn.Sequential(nn.Conv2d(3, 64, 2,2),                                                                     
                                   nn.BatchNorm2d(64))
        self.seq_2 = nn.Sequential(nn.Conv2d(64, 128, 2,1), nn.GELU(),
                                   nn.MaxPool2d(2,2, ), 
                                   nn.BatchNorm2d(128))
        self.seq_3 = nn.Sequential(nn.Conv2d(128, 256, 2,1, padding = "same"), nn.GELU(),
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(256, ))
        self.seq_4 = nn.Sequential(nn.Conv2d(256, 512, 2,1, padding = "same"), nn.GELU(),
                                    
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(512, ))
        self.seq_5 = nn.Sequential(nn.Conv2d(512, 128, 2,1, padding = "same"), nn.GELU(),
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(128, ))
        
        self.f = nn.Flatten()
        self.dense = nn.Linear(4608, 100)
    def forward(self, x):
        x = self.seq_1(x)
        x = self.seq_2(x)
        x = self.seq_3(x)
        x = self.seq_4(x)
        x = self.f(x)
        x = self.dense(x)
        return x

In [40]:
model_ = VGG5()
model = torch.nn.DataParallel(model_)
model.to(0)

DataParallel(
  (module): VGG5(
    (seq_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
      (1): GELU(approximate='none')
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
      (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): MaxPool2

In [41]:
model(torch.randn(1,3,32,32)).shape

torch.Size([1, 100])

In [42]:
def loss(x,y):
    return nn.CrossEntropyLoss()(model(x),y)

In [45]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000001)

In [ ]:
t = 0
while True:
    L = []
    loss__ = []
    model.train()

    for i, batch in enumerate(train_set):
        x, y = batch[0].cuda(), batch[1].cuda()
    
        loss_ = loss(x,y)
        loss_.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss__.append(loss_.item())

    model.eval()
    with torch.no_grad():
            
        for j, batch in enumerate(test_set):
            cond = 1
            if cond: ### here we do reservoir sampling
                x, y = batch[0].cuda(), batch[1].cuda()
                L.append(sum(torch.argmax(model(x), dim = 1) == y).item())
            else:
                continue
        
    print(t, sum(L)/len(L), sum(loss__)/len(loss__))
    t += 1

0 56.68354430379747 2.088542774815084
1 57.77215189873418 2.1051719347229394
2 55.75949367088607 2.0889554386553555
3 57.55696202531646 2.0969594387752015
4 57.12658227848101 2.091485524421458
5 56.77215189873418 2.084475754167113
6 56.0253164556962 2.089185520511149
7 54.075949367088604 2.087197310174518
8 57.78481012658228 2.087963814930538
9 57.49367088607595 2.078727123682456
10 57.20253164556962 2.086736709870341
11 57.78481012658228 2.08079997962698
12 58.063291139240505 2.087153507010711
13 57.35443037974684 2.0742868703344595
14 57.87341772151899 2.075040643782262
15 56.74683544303797 2.0790855344908925
16 57.91139240506329 2.0723780103961524
17 57.949367088607595 2.0744589777553784
18 58.44303797468354 2.071952709456539
19 56.87341772151899 2.081192556854404
20 57.88607594936709 2.0761637041330947
21 58.10126582278481 2.075338909083315
22 57.45569620253165 2.0660709325614794
23 57.67088607594937 2.0644180146629547
24 57.278481012658226 2.0732933063336345
25 57.63291139240506 2